In [55]:
from GraphTsetlinMachine.graphs import Graphs
import numpy as np


In [56]:
def getConnectionsOld_bad(size, index):
    x = index % size
    y = index // size

    # Define out-of-bounds placeholders
    out_of_bounds_right = "out_of_bounds_right"
    out_of_bounds_left = "out_of_bounds_left"
    out_of_bounds_down = "out_of_bounds_down"
    out_of_bounds_up = "out_of_bounds_up"

    connections = []
    directions = []

    # Up
    if y > 0:
        connections.append(x + (y - 1) * size)  # Directly above
        directions.append("up")

        # Diagonal up-left for even rows, up-right for odd rows
        if y % 2 == 0:  # Even row
            if x > 0:  # Only add if x > 0 (to avoid out-of-bound left cases)
                connections.append((x - 1) + (y - 1) * size)  # Diagonal up-left
                directions.append("up_left")
        else:  # Odd row
            if x < size - 1:  # Only add if x < size-1 (to avoid out-of-bound right cases)
                connections.append((x + 1) + (y - 1) * size)  # Diagonal up-right
                directions.append("up_right")
    else:
        connections.append(out_of_bounds_up)
        directions.append("up")

    # Left
    if x > 0:
        connections.append((x - 1) + y * size)
        directions.append("left")
    else:
        connections.append(out_of_bounds_left)
        directions.append("left")

    # Right
    if x < size - 1:
        connections.append((x + 1) + y * size)
        directions.append("right")
    else:
        connections.append(out_of_bounds_right)
        directions.append("right")

    # Down
    if y < size - 1:
        connections.append(x + (y + 1) * size)  # Directly below
        directions.append("down")

        # Diagonal down-left for even rows, down-right for odd rows
        if y % 2 == 0:  # Even row
            if x > 0:  # Only add if x > 0 (to avoid out-of-bound left cases)
                connections.append((x - 1) + (y + 1) * size)  # Diagonal down-left
                directions.append("down_left")
        else:  # Odd row
            # Correct logic for down-right: Check grid boundary conditions
            if x < size - 1 and (y + 1) * size + (x + 1) < size * size:  
                # Only add if there is a valid neighbor (within bounds)
                connections.append((x + 1) + (y + 1) * size)  # Diagonal down-right
                directions.append("down_right")
    else:
        connections.append(out_of_bounds_down)
        directions.append("down")

    return connections, directions


In [57]:
from numba import jit
import pickle

@jit
def getConnections(size, x):
        connections = []
        directions = []

        max_index = size**2-1
        right = max_index + 1
        left = max_index + 2
        down = max_index + 3
        up = max_index + 4

        if x >= size:#If x is not on top row
            connections.append(x-size)#Append top left
            directions.append("up_left")
            if x%size != size-1:#If x is not on right column
                connections.append(x-size+1) #Append top right
                directions.append("up_right")
        else:
            connections.append(up)
            directions.append("Up")
        
        if x%size != 0:#If x is not on left column
            connections.append(x-1) #Append left
            directions.append("left")
        else:
            connections.append(left)
            directions.append("Left")

        if x%size != size-1:#If x is not on right column
            connections.append(x+1)#Append right
            directions.append("right")
        else:
            connections.append(right)
            directions.append("Right")
        
        if x < (size**2)-size: #If x is not on bottom row
            connections.append(x+size)#Append bottom right
            directions.append("down_right")
            if x%size != 0:#If x is not on left column
                connections.append(x+size-1)#Append bottom left
                directions.append("down_left")
        else:
            directions.append("Down")
            connections.append(down)

        
            
        return connections, directions
            
        #if y > 0:
        #    connections.append([x, y-1])
        #    if x < self.size-1:
        #        connections.append([x+1, y-1])
        #if x > 0:
        #    connections.append([x-1, y])
        #if x < self.size-1:
        #    connections.append([x+1, y])
        #if y < self.size-1:
        #    connections.append([x, y+1])
        #    if x > 0:
        #        connections.append([x-1, y+1])


def getConnectionsExhaustive(size):
    fileConnections = f"data/connections_{size}.pkl"
    fileDirections = f"data/directions_{size}.pkl"
    try:
        with open(fileConnections, 'rb') as fCon:
            allConnections = pickle.load(fCon)
        fCon.close()
        with open(fileDirections, "rb") as fDir:
            allDirections = pickle.load(fDir)
        fDir.close()
    except:
        allConnections = []
        allDirections = []
        for i in range(size**2):
            conn, dir = getConnections(size,i)
            allConnections.append(conn)
            allDirections.append(dir)
        with open(fileConnections, "wb") as fCon:
            pickle.dump(allConnections,fCon)
        fCon.close()
        with open(fileDirections, "wb") as fDir:
            pickle.dump(allDirections, fDir)
        fDir.close()
    return allConnections, allDirections


def getConnectionsOnlyColor(size,x,board, directional = False):
        connections = []
        directions = []

        if not directional:
            if board[x] == 0:
                return [],[]


        max_index = size**2-1
        right = max_index + 1
        left = max_index + 2
        down = max_index + 3
        up = max_index + 4

        if x >= size:#If x is not on top row
            if board[x-size] == 1:
                connections.append(x-size)#Append top left
                directions.append("up_left_red")
            if board[x-size] == -1:
                connections.append(x-size)#Append top left
                directions.append("up_left_blue")
            if x%size != size-1:#If x is not on right column
                if board[x-size+1] == 1:
                    connections.append(x-size+1) #Append top right
                    directions.append("up_right_red")
                elif board[x-size+1] == -1:
                    connections.append(x-size+1) #Append top right
                    directions.append("up_right_blue")
        else:
            connections.append(up)
            directions.append("Up")
        
        if x%size != 0:#If x is not on left column
            if board[x-1] == 1:
                directions.append("left_red")
                connections.append(x-1) #Append left
            elif board[x-1] == -1:
                connections.append(x-1) #Append left
                directions.append("left_blue")
        else:
            connections.append(left)
            directions.append("Left")

        if x%size != size-1:#If x is not on right column
            if board[x+1] == 1:
                connections.append(x+1)#Append right
                directions.append("right_red")
            elif board[x+1] == -1:
                connections.append(x+1)#Append right
                directions.append("right_blue")
        else:
            connections.append(right)
            directions.append("Right")
        
        if x < (size**2)-size: #If x is not on bottom row
            if board[x+size] == 1:
                connections.append(x+size)#Append bottom right
                directions.append("down_right_red")
            elif board[x+size] == -1:
                connections.append(x+size)#Append bottom right
                directions.append("down_right_blue")
            if x%size != 0:#If x is not on left column
                if board[x+size-1] == 1:
                    connections.append(x+size-1)#Append bottom left
                    directions.append("down_left_red")
                elif board[x+size-1] == -1:
                    connections.append(x+size-1)#Append bottom left
                    directions.append("down_left_blue")
        else:
            directions.append("Down")
            connections.append(down)

        return connections, directions

def getConnectionsInterColor(size,x,board, directional = False):
        connections = []
        directions = []
        
        c = board[x]

        if not directional:
            if c == 0:
                return [],[]

        max_index = size**2-1
        right = max_index + 1
        left = max_index + 2
        down = max_index + 3
        up = max_index + 4

        if x >= size:#If x is not on top row
            if board[x-size] == c:
                connections.append(x-size)#Append top left
                directions.append("up_left")
            if x%size != size-1:#If x is not on right column
                if board[x-size+1] == c:
                    connections.append(x-size+1) #Append top right
                    directions.append("up_right")
        else:
            connections.append(up)
            directions.append("Up")
        
        if x%size != 0:#If x is not on left column
            if board[x-1] == c:
                connections.append(x-1) #Append left
                directions.append("left")
        else:
            connections.append(left)
            directions.append("Left")

        if x%size != size-1:#If x is not on right column
            if board[x+1] == c:
                connections.append(x+1)#Append right
                directions.append("right")
        else:
            connections.append(right)
            directions.append("Right")
        
        if x < (size**2)-size: #If x is not on bottom row
            if board[x+size] == c:
                connections.append(x+size)#Append bottom right
                directions.append("down_right")
            if x%size != 0:#If x is not on left column
                if board[x+size-1] == c:
                    connections.append(x+size-1)#Append bottom left
                    directions.append("down_left_red")
        else:
            directions.append("Down")
            connections.append(down)

        return connections, directions

@jit
def getConnectionsColor(size,x,board):
        connections = []
        directions = []

        max_index = size**2-1
        right = max_index + 1
        left = max_index + 2
        down = max_index + 3
        up = max_index + 4

        if x >= size:#If x is not on top row
            connections.append(x-size)#Append top left
            if board[x-size] == 1:
                directions.append("up_left_red")
            if board[x-size] == -1:
                directions.append("up_left_blue")
            else:
                directions.append("up_left_empty")
            if x%size != size-1:#If x is not on right column
                connections.append(x-size+1) #Append top right
                if board[x-size+1] == 1:
                    directions.append("up_right_red")
                elif board[x-size+1] == -1:
                    directions.append("up_right_blue")
                else:
                    directions.append("up_right_empty")
        else:
            connections.append(up)
            directions.append("Up")
        
        if x%size != 0:#If x is not on left column
            connections.append(x-1) #Append left
            if board[x-1] == 1:
                directions.append("left_red")
            elif board[x-1] == -1:
                directions.append("left_blue")
            else:
                directions.append("left_empty")
        else:
            connections.append(left)
            directions.append("Left")

        if x%size != size-1:#If x is not on right column
            connections.append(x+1)#Append right
            if board[x+1] == 1:
                directions.append("right_red")
            elif board[x+1] == -1:
                directions.append("right_blue")
            else:
                directions.append("right_empty")
        else:
            connections.append(right)
            directions.append("Right")
        
        if x < (size**2)-size: #If x is not on bottom row
            connections.append(x+size)#Append bottom right
            if board[x+size] == 1:
                directions.append("down_right_red")
            elif board[x+size] == -1:
                directions.append("down_right_blue")
            else:
                directions.append("down_right_empty")
            if x%size != 0:#If x is not on left column
                connections.append(x+size-1)#Append bottom left
                if board[x+size-1] == 1:
                    directions.append("down_left_red")
                elif board[x+size-1] == -1:
                    directions.append("down_left_blue")
                else:
                    directions.append("down_left_empty")
        else:
            directions.append("Down")
            connections.append(down)

        return connections, directions

In [58]:
# read the data from data/hex_games_1_000_000_size_7.csv
data = np.genfromtxt('data/11x11_200000_40_0.csv', delimiter=',', dtype=np.int32, skip_header=1, max_rows=400000)
#data = np.genfromtxt('data/hex_games_8x8_34.csv', delimiter=',', dtype=np.int32, skip_header=1, max_rows=400000)
data


array([[ 1,  0, -1, ...,  0,  1,  1],
       [ 0,  0,  0, ...,  0,  0,  1],
       [-1,  1,  0, ...,  1,  0,  1],
       ...,
       [ 0, -1,  1, ...,  1,  0,  0],
       [-1,  1, -1, ...,  1,  1,  0],
       [ 1,  1, -1, ...,  0,  1,  1]])

In [59]:
#sumsamples = 30
#np.random.shuffle(data)
#data = data[:sumsamples]
data = np.unique(data,axis=0)
print(len(data))
# duplicate the data 100 times
#data = np.repeat(data, 1000, axis=0)

139606


In [60]:
data.shape

(139606, 123)

In [61]:
# separate the last column from the rest of the data
X_data = data[:, :-2]
Y_data = data[:, -1]
X_data[0].shape

(121,)

In [62]:
Y_data.mean()

0.49888256951706944

In [63]:
from sklearn.model_selection import train_test_split
# train test splitt

X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)
board_size = int(np.sqrt(X_data.shape[1]))
number_of_nodes = board_size*board_size
    


In [64]:
def makeGraphs(boards, connected_diff = True, connected_same = False, temp_encoding = True, edge_nodes = False, connectionsColor = True, connectionsOnlyColor = False, connectionsInterColor = False, directionalEncodingEmpty = False, gap = False, other_graph=None):
    
    if connectionsInterColor:
        connectionsOnlyColor = False
        connectionsColor = False
    elif connectionsOnlyColor:
        connectionsColor = False


    hypervector_bits = 2
    hypervector_size = 128

    board_size = int(np.sqrt(boards.shape[1]))
    number_of_nodes = board_size**2
    
    symbol_names = ['RED', 'BLUE']
    edges = ['UP', 'DOWN', 'RIGHT','LEFT']
    edge_directions = ["Up", "Down", "Right", "Left"]

    if edge_nodes:
        symbol_names.extend(edges)

    for i in range(board_size):
        symbol_names.append(f'ROW_{i}')
        symbol_names.append(f'COL_{i}')

    #symbol_names.append(f"BLUE_NEIGHBOURS") 
    #symbol_names.append(f"RED_NEIGHBOURS")
    if connected_diff:
        symbol_names.append("CONNECTED_BLUE")
        symbol_names.append("CONNECTED_RED")
    elif connected_same:
        symbol_names.append("CONNECTED")
    if gap:
        symbol_names.extend(["GAP_BLUE", "GAP_RED"])
    #symbol_names.extend(["LEFT_EMPTY", "RIGHT_EMPTY", "UP_EMPTY", "DOWN_EMPTY"])

    max_index = number_of_nodes-1

    right_index = max_index + 1
    left_index = max_index + 2
    down_index = max_index + 3
    up_index = max_index + 4

    edge_indexes = [right_index,left_index,down_index,up_index]


    if other_graph:
        print("Test")
        graphs = Graphs(boards.shape[0],init_with=other_graph)
    else:
        graphs = Graphs(boards.shape[0],symbols=symbol_names, hypervector_size=hypervector_size, hypervector_bits=hypervector_bits, double_hashing = False)

    for graph_id in range(boards.shape[0]):
        if edge_nodes:
            graphs.set_number_of_graph_nodes(graph_id, number_of_nodes+4)
        else:
            graphs.set_number_of_graph_nodes(graph_id, number_of_nodes)
    graphs.prepare_node_configuration()
    # add up, down, left, right nodes
    #X_trainfor graph_id in range(X_train.shape[0]):




    # add up, down, left, right nodes




    # Add nodes to each graph
    for graph_id in range(boards.shape[0]):
        for node_id in range(number_of_nodes):
            if connectionsInterColor:
                if edge_nodes:
                    nr_neighbours = len(getConnectionsInterColor(board_size, node_id,boards[graph_id],directionalEncodingEmpty)[0])
                else:
                    nr_neighbours = len([i for i in getConnectionsInterColor(board_size,node_id,boards[graph_id],directionalEncodingEmpty)[0] if i not in edge_indexes])
                graphs.add_graph_node(graph_id, node_id, nr_neighbours) 
            elif connectionsOnlyColor:
                if edge_nodes:
                    nr_neighbours = len(getConnectionsOnlyColor(board_size, node_id,boards[graph_id],directionalEncodingEmpty)[0])
                else:
                    nr_neighbours = len([i for i in getConnectionsOnlyColor(board_size,node_id,boards[graph_id],directionalEncodingEmpty)[0] if i not in edge_indexes])
            else:
                if edge_nodes:
                    nr_neighbours = len(getConnections(board_size, node_id)[0])
                else:
                    nr_neighbours = len([i for i in getConnections(board_size,node_id)[0] if i not in edge_indexes])
            graphs.add_graph_node(graph_id, node_id, nr_neighbours) 
        if edge_nodes:
            graphs.add_graph_node(graph_id, right_index, board_size)
            graphs.add_graph_node(graph_id, left_index, board_size)
            graphs.add_graph_node(graph_id, down_index,board_size)
            graphs.add_graph_node(graph_id, up_index, board_size)

    graphs.prepare_edge_configuration()

    allConnections, allDirections = getConnectionsExhaustive(board_size)
    for graph_id in range(boards.shape[0]):
        for node_id in range(number_of_nodes+4):
            edge_type = 0
            if node_id<number_of_nodes:
                
                if connectionsInterColor:
                    neighbors, directions = getConnectionsInterColor(board_size, node_id,boards[graph_id],directionalEncodingEmpty)

                elif connectionsOnlyColor:
                    neighbors, directions = getConnectionsOnlyColor(board_size, node_id,boards[graph_id],directionalEncodingEmpty)

                elif connectionsColor:
                    neighbors, directions = getConnectionsColor(board_size,node_id,boards[graph_id])
                else:
                    neighbors, directions = getConnections(board_size, node_id)

                if not edge_nodes:
                    neighbors = [i for i in neighbors if i not in edge_indexes]
                    directions = [i for i in directions if i not in edge_directions]

                for neighbor_id,dir in zip(neighbors,directions):
                    graphs.add_graph_node_edge(graph_id, node_id, neighbor_id,dir)

            

            

                node_value = boards[graph_id, node_id]

                redCount = 0
                blueCount = 0

                for i in neighbors:
                    if i<board_size**2:
                        if boards[graph_id,i] == 1:
                            redCount += 1
                        elif boards[graph_id,i] == -1:
                            blueCount += 1
                
                #if node_value != 0:
                #    if redCount > 1:
                #        graphs_train.add_graph_node_property(graph_id,node_id, f"RED_NEIGHBOURS")
                #    if blueCount > 1:
                #        graphs_train.add_graph_node_property(graph_id,node_id, f"BLUE_NEIGHBOURS")

                    if node_value == 1:
                        graphs.add_graph_node_property(graph_id, node_id, 'RED')
                        if connected_diff:
                            if (redCount > 1) or (redCount != 0 and node_id >= (board_size**2)-board_size) or (redCount != 0 and node_id<board_size):
                                graphs.add_graph_node_property(graph_id, node_id, "CONNECTED_RED")
                            
                    elif node_value == -1:
                        graphs.add_graph_node_property(graph_id, node_id, 'BLUE')
                        if connected_diff:
                            if (blueCount > 1) or (blueCount != 0 and node_id%board_size == 0) or (blueCount != 0 and node_id%board_size == board_size-1):
                                graphs.add_graph_node_property(graph_id, node_id, "CONNECTED_BLUE")

                if node_value == 0:
                    if gap:
                        if (blueCount > 1) or (blueCount != 0 and node_id%board_size == 0) or (blueCount != 0 and node_id%board_size == board_size-1):
                            graphs.add_graph_node_property(graph_id, node_id, "GAP_BLUE")
                        if (redCount > 1) or (redCount != 0 and node_id >= (board_size**2)-board_size) or (redCount != 0 and node_id<board_size):
                            graphs.add_graph_node_property(graph_id, node_id, "GAP_RED")

                    #if (blueCount > 1) or (blueCount != 0 and left_index in neighbors) or (blueCount != 0 and right_index in neighbors):
                    #    graphs_train.add_graph_node_property(graph_id, node_id, "CONNECTED_BLUE")
                    #if (redCount > 1) or (redCount != 0 and down_index in neighbors) or (redCount != 0 and up_index in neighbors):
                    #    graphs_train.add_graph_node_property(graph_id, node_id, "CONNECTED_RED")
                    #graphs_train.add_graph_node_property(graph_id, node_id, 'EMPTY')
                    #if left_index in neighbors:
                    #    graphs_train.add_graph_node_property(graph_id, node_id, 'LEFT_EMPTY')
                    #elif right_index in neighbors:
                    #    graphs_train.add_graph_node_property(graph_id, node_id, 'RIGHT_EMPTY')
                    #if up_index in neighbors:
                    #    graphs_train.add_graph_node_property(graph_id, node_id, 'UP_EMPTY')
                    #elif down_index in neighbors:
                    #    graphs_train.add_graph_node_property(graph_id, node_id, 'DOWN_EMPTY')
                        

                    
                row = node_id // board_size
                col = node_id % board_size
                
                    
        
                #    graphs_train.add_graph_node_property(graph_id, node_id, f'ROW_{row}COL_{col}')
                if temp_encoding:
                    for r in range(row+1):
                        graphs.add_graph_node_property(graph_id, node_id, f'ROW_{r}')
                    for c in range(col+1):
                        graphs.add_graph_node_property(graph_id, node_id, f'COL_{c}')
                else:
                        graphs.add_graph_node_property(graph_id, node_id, f'ROW_{row}')
                        graphs.add_graph_node_property(graph_id, node_id, f'COL_{col}')
            if edge_nodes:
                if node_id == right_index:
                    neighbors = [i for i in range(board_size-1,board_size*board_size,board_size)]
                    edge_type = 'Right'
                    for neighbor_id in neighbors:
                        graphs.add_graph_node_edge(graph_id, node_id, neighbor_id,edge_type)
                    graphs.add_graph_node_property(graph_id, node_id, 'RIGHT')
                if node_id == left_index:
                    neighbors = [i for i in range(0,board_size*board_size,board_size)]
                    edge_type = 'Left'
                    for neighbor_id in neighbors:
                        graphs.add_graph_node_edge(graph_id, node_id, neighbor_id,edge_type)
                    graphs.add_graph_node_property(graph_id, node_id, 'LEFT')
                if node_id == down_index:
                    neighbors = [i for i in range(board_size*board_size-board_size,board_size*board_size,1)]
                    edge_type = 'Down'
                    for neighbor_id in neighbors:
                        graphs.add_graph_node_edge(graph_id, node_id, neighbor_id,edge_type)
                    graphs.add_graph_node_property(graph_id, node_id, 'DOWN')
                if node_id == up_index:
                    neighbors = [i for i in range(board_size)]
                    edge_type = 'Up'
                    for neighbor_id in neighbors:
                        graphs.add_graph_node_edge(graph_id, node_id, neighbor_id,edge_type)
                    graphs.add_graph_node_property(graph_id, node_id, 'UP')

    graphs.encode()
    return graphs

In [65]:


import pickle

def makeGraphWrapper(X_train, Y_train, X_test, Y_test, moves, arr):

    if 0 in arr:
        graphs_train = makeGraphs(X_train)
        graphs_test = makeGraphs(X_test,other_graph=graphs_train)

        with open(f'pickled_graphs/default_{moves}.pkl', 'wb') as f:
            pickle.dump((graphs_train, graphs_test, X_train, Y_train, X_test, Y_test), f)
        f.close()

    if 1 in arr:
        graphs_train = makeGraphs(X_train, edge_nodes=True)
        graphs_test = makeGraphs(X_test, edge_nodes=True,other_graph=graphs_train)

        with open(f'pickled_graphs/edgenodes_{moves}.pkl', 'wb') as f:
            pickle.dump((graphs_train, graphs_test, X_train, Y_train, X_test, Y_test), f)
        f.close()

    if 2 in arr:
        graphs_train = makeGraphs(X_train, connected_diff=False)
        graphs_test = makeGraphs(X_test, connected_diff=False,other_graph=graphs_train)

        with open(f'pickled_graphs/noConnected_{moves}.pkl', 'wb') as f:
            pickle.dump((graphs_train, graphs_test, X_train, Y_train, X_test, Y_test), f)
        f.close()


    if 3 in arr:
        graphs_train = makeGraphs(X_train, connectionsOnlyColor=True)
        graphs_test = makeGraphs(X_test, connectionsOnlyColor=True,other_graph=graphs_train)

        with open(f'pickled_graphs/undirectedConnectionsOnlyColor_{moves}.pkl', 'wb') as f:
            pickle.dump((graphs_train, graphs_test, X_train, Y_train, X_test, Y_test), f)
        f.close()

    if 4 in arr:
        graphs_train = makeGraphs(X_train, connectionsOnlyColor=True, directionalEncodingEmpty=True)
        graphs_test = makeGraphs(X_test, connectionsOnlyColor=True, directionalEncodingEmpty=True,other_graph=graphs_train)

        with open(f'pickled_graphs/directedConnectionsOnlyColor_{moves}.pkl', 'wb') as f:
            pickle.dump((graphs_train, graphs_test, X_train, Y_train, X_test, Y_test), f)
        f.close()

    if 5 in arr:
        graphs_train = makeGraphs(X_train, connectionsInterColor=True)
        graphs_test = makeGraphs(X_test, connectionsInterColor=True,other_graph=graphs_train)

        with open(f'pickled_graphs/undirectedConnectionsInterColor_{moves}.pkl', 'wb') as f:
            pickle.dump((graphs_train, graphs_test, X_train, Y_train, X_test, Y_test), f)
        f.close()

    if 6 in arr:
        graphs_train = makeGraphs(X_train, connectionsInterColor=True, directionalEncodingEmpty=True)
        graphs_test = makeGraphs(X_test, connectionsInterColor=True, directionalEncodingEmpty=True, other_graph=graphs_train)

        with open(f'pickled_graphs/directedConnectionsInterColor_{moves}.pkl', 'wb') as f:
            pickle.dump((graphs_train, graphs_test, X_train, Y_train, X_test, Y_test), f)
        f.close()

    if 7 in arr:
        graphs_train = makeGraphs(X_train,temp_encoding=False)
        graphs_test = makeGraphs(X_test,temp_encoding=False,other_graph=graphs_train)

        with open(f'pickled_graphs/noTempEncoding_{moves}.pkl', 'wb') as f:
            pickle.dump((graphs_train, graphs_test, X_train, Y_train, X_test, Y_test), f)
        f.close()
    
    if 8 in arr:
        graphs_train = makeGraphs(X_train,gap=True)
        graphs_test = makeGraphs(X_test, gap=True,other_graph=graphs_train)

        with open(f'pickled_graphs/gapEncoding_{moves}.pkl', 'wb') as f:
            pickle.dump((graphs_train, graphs_test, X_train, Y_train, X_test, Y_test), f)
        f.close()

In [66]:
makeGraphWrapper(X_train,Y_train,X_test,Y_test,0,[0,2])

data2 = np.genfromtxt('data/11x11_200000_40_2.csv', delimiter=',', dtype=np.int32, skip_header=1, max_rows=400000)
data2 = np.unique(data2,axis=0)
print("len data 2", len(data2))

X_data2 = data2[:, :-2]
Y_data2 = data2[:, -1]

X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X_data2, Y_data2, test_size=0.2, random_state=42)

makeGraphWrapper(X_train2,Y_train2,X_test2,Y_test2,2,[0,2])

data5 = np.genfromtxt('data/11x11_200000_40_5.csv', delimiter=',', dtype=np.int32, skip_header=1, max_rows=400000)
data5 = np.unique(data5,axis=0)
print("len data 5", len(data5))

X_data5 = data5[:, :-2]
Y_data5 = data5[:, -1]

X_train5, X_test5, Y_train5, Y_test5 = train_test_split(X_data5, Y_data5, test_size=0.2, random_state=42)

makeGraphWrapper(X_train5,Y_train5,X_test5,Y_test5,5,[0,2])

Test
Test
len data 2 139606
Test
Test
len data 5 139606
Test
Test


In [67]:
graphs_train = makeGraphs(X_train, connected_diff=False)

KeyboardInterrupt: 

In [ ]:

graphs_test = makeGraphs(X_test, connected_diff=False, other_graph=graphs_train)


Test


In [18]:
# pickle dump all the  (graphs_train, graphs_test, X_train, Y_train, X_test, Y_test)
import pickle
with open('data.pkl', 'wb') as f:
    pickle.dump((graphs_train, graphs_test, X_train, Y_train, X_test, Y_test), f)


In [36]:
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine

In [47]:
number_of_nodes = board_size*board_size
number_of_clauses = 3900
T = number_of_clauses*1.5971422963103452
depth = 1
s = 1.051723993754933015*2
message_size = 64
message_bits = 3
max_included_literals = 59
number_of_states = 200
epochs = 40

In [48]:
# Verify sizes and shapes
print(f"Number of Graphs: {graphs_train.number_of_graphs}")
print(f"Number of Nodes: {graphs_train.number_of_nodes}")
print(f"X Shape: {graphs_train.X.shape}")
print(f"Edge Array Shape: {graphs_train.edge.shape}")

# Check indices
print(f"Node Index Array: {graphs_train.node_index}")
print(f"Edge Index Array: {graphs_train.edge_index}")


Number of Graphs: 111684
Number of Nodes: 13513764
X Shape: (13513764, 8)
Edge Array Shape: (12027932, 2)
Node Index Array: [       0      121      242 ... 13513401 13513522 13513643]
Edge Index Array: [       0        0        0 ... 12027930 12027931 12027932]


In [49]:
tm = MultiClassGraphTsetlinMachine(
    number_of_clauses, T, s, depth=depth, message_size = message_size,
    message_bits = message_bits, number_of_state_bits = 8, max_included_literals = max_included_literals,
    grid=(16*13,1,1),
  	block=(128,1,1)

)


Initialization of sparse structure.


In [50]:
Y_test.mean()

0.4985316238091827

In [51]:


#torch.cuda.empty_cache()<
#torch.cuda.reset_max_memory_allocated() # Clear GPU memory
for i in range(epochs):
    tm.fit(graphs_train, Y_train, epochs=1, incremental=True)
  
    result_test = 100 * ( tm.predict(graphs_test) == Y_test).mean()
    result_train = 100 * (tm.predict(graphs_train) == Y_train).mean()
    
    print("#%d Testing Accuracy: %.2f%% Training Accuracy: %.2f%%" % (i+1, result_test, result_train))

    if result_test == 100:
        break
    
   
  

c:\Users\isakk\anaconda3\envs\NorBertTest1\lib\site-packages\GraphTsetlinMachine\tm.py:195: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod_prepare = SourceModule(parameters + kernels.code_header + kernels.code_prepare, no_extern_c=True)
c:\Users\isakk\anaconda3\envs\NorBertTest1\lib\site-packages\GraphTsetlinMachine\tm.py:202: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod_update = SourceModule(parameters + kernels.code_header + kernels.code_update, no_extern_c=True)
c:\Users\isakk\anaconda3\envs\NorBertTest1\lib\site-packages\GraphTsetlinMachine\tm.py:209: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu

  mod_evaluate = SourceModule(parameters + kernels.code_header + kernels.code_evaluate, no_extern_c=True)


#1 Testing Accuracy: 94.16% Training Accuracy: 94.11%
#2 Testing Accuracy: 95.88% Training Accuracy: 95.87%
#3 Testing Accuracy: 95.93% Training Accuracy: 95.94%
#4 Testing Accuracy: 96.15% Training Accuracy: 95.99%
#5 Testing Accuracy: 95.29% Training Accuracy: 95.44%
#6 Testing Accuracy: 95.81% Training Accuracy: 95.83%
#7 Testing Accuracy: 96.22% Training Accuracy: 96.24%
#8 Testing Accuracy: 96.26% Training Accuracy: 96.45%
#9 Testing Accuracy: 96.83% Training Accuracy: 96.92%
#10 Testing Accuracy: 96.74% Training Accuracy: 96.69%
#11 Testing Accuracy: 96.83% Training Accuracy: 96.89%
#12 Testing Accuracy: 97.12% Training Accuracy: 97.17%
#13 Testing Accuracy: 96.89% Training Accuracy: 96.94%
#14 Testing Accuracy: 97.19% Training Accuracy: 97.26%
#15 Testing Accuracy: 97.04% Training Accuracy: 97.08%
#16 Testing Accuracy: 96.46% Training Accuracy: 96.43%
#17 Testing Accuracy: 96.53% Training Accuracy: 96.66%
#18 Testing Accuracy: 95.68% Training Accuracy: 95.75%
#19 Testing Accurac

KeyboardInterrupt: 

In [ ]:
tm.score(graphs_test)

array([[  96527,  -95964],
       [-170927,  172050],
       [ 171746, -170165],
       ...,
       [-246102,  246005],
       [-185436,  184495],
       [ 192271, -190743]])

In [44]:
result_test

100.0

In [45]:
result_train

100.0

In [46]:
predictions = tm.predict(graphs_test)

In [47]:
predictions.mean()

0.4931959175505303

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(Y_test,predictions))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5065
           1       1.00      1.00      1.00      4929

    accuracy                           1.00      9994
   macro avg       1.00      1.00      1.00      9994
weighted avg       1.00      1.00      1.00      9994



In [49]:
confusion_matrix(Y_test,predictions)

array([[5065,    0],
       [   0, 4929]], dtype=int64)

In [50]:
graphs_train.hypervectors

array([[ 38,  51],
       [ 18, 109],
       [ 63,  39],
       [125,  21],
       [106,  25],
       [ 73,  26],
       [ 14,  52],
       [ 40,  64],
       [ 68,  46],
       [ 59,  73],
       [127,   1],
       [107,  58],
       [ 23,  30],
       [ 22, 108],
       [ 41,  44],
       [ 74,  54],
       [ 42,  40],
       [ 37, 112]], dtype=uint32)

In [ ]:
weights = tm.get_state()[1].reshape(2, -1)
for i in range(tm.number_of_clauses):
        print("Clause #%d W:(%d %d)" % (i, weights[0,i], weights[1,i]), end=' ')
        l = []
        for k in range(hypervector_size * 2):
            if tm.ta_action(0, i, k):
                if k < hypervector_size:
                    l.append("x%d" % (k))
                else:
                    l.append("NOT x%d" % (k - hypervector_size))
        print(" AND ".join(l))


Clause #0 W:(-69 64) 

NameError: name 'hypervector_size' is not defined